# An exmple of using GPT Functions calling

"Function calling" is a quite reliable approach to extracting structured, deterministic information from large language models like GPT-4. We can now describe functions to GPT-4-0613 and GPT-3.5-turbo-0613 models and have the model automatically choose to create a JSON object output with parameters to call those functions. It's a novel approach to establishing dependable connections between GPT and third-party applications and APIs. OpenAI fine-tuned these models to recognize when a function should be invoked (based on user input) and return JSON that follows the function signature. 

**In this example, we will use free to use weather API from openweathermap.org to get the current weather information for a place requested by the user.**

Install PYOWM, a Python wrapper around OpenWeatherMap web APIs to get weather data.

In [ ]:
#!pip install pyowm

Import the usual suspects. :)

In [1]:
import openai
import json

#import to use OpenWeatherMap API for Weather Function example
import pyowm

Import the API Keys. 
<div class="alert alert-block alert-info">Note: OpenWeatherMap has free API keys with limited functionalities. Use the free one.</div>

In [2]:
# Load API Keys from files
openai.api_key = open("openai_api.key", "r").read().strip("\n")
owm_api_key=open("owm_api.key", "r").read().strip("\n")  

<div class="alert alert-block alert-warning">This is a placeholder cell to test out various functions of the OWM API. Feel free to explore it. :)</div>

In [ ]:
#Test API call before punching it into GPT ... delete cell later after creating real function below. 
from pyowm.owm import OWM
#from pyowm.utils import timestamps
owm = OWM(owm_api_key)

mgr = owm.weather_manager()
observation = mgr.weather_at_place('Arizona,US')
location_temperature = observation.weather.temperature('fahrenheit')['temp']
weathernow = observation.weather.detailed_status 
print(location_temperature)
print (weathernow)

#daily_forecaster = mgr.forecast_at_place('Arizona,US', 'daily')    # looks like free API has its limitations. :)
#tomorrow = timestamps.tomorrow()                                   # datetime object for tomorrow
#weather = daily_forecaster.get_weather_at(tomorrow)                # the weather forecast
#print(weather)

#### User Input:

In [3]:
userInput = "What's the weather like in Jamaica?"

Before Function calls were a possibility, we will do the following to get a completion:

In [4]:
completion = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[{"role": "user", "content": userInput}],
)
print(completion.choices[0].message.content)

As an AI, I don't have real-time data. However, Jamaica typically has a tropical climate with hot and humid weather, with little variation throughout the year. Temperatures average between 80-90°F (27-32°C) during the day and around 70°F (21°C) at night. It can rain frequently, but the island also enjoys plenty of sunshine. The rainier months are usually May and October to November. Always check a reliable weather forecast before planning your visit.


**Function calling lets us overcome GPT's inherent limitations, in this case real time information, by allowing us to invoke function(s) to provide the information that the user requested by supplementing GPT's inherent capabilities.**

<div class="alert alert-block alert-info">
Here's what we will do.
    
1. Define the function to get current weather of a place requested by the user. In our example here, it's Arizona. 
2. Send the conversation and all available functions to GPT (yes, you can send more than one functions to the LLM).
3. Function can be set as auto for LLM to decide if the context requires it to make the call. In this example, we will be explicit and ask LLM to make the call.
4. Check if GPT wanted to call a function.
5. Send the info on the function call and function response to GPT get the final completion from GPT with the function call results.
6. Print the final completion.
</div>

Define the function that gets the current weather of a location from https://openweathermap.org/

In [5]:
def get_current_weather(location, unit):
    """Get the current weather in a given location"""
    from pyowm.owm import OWM
    owm = OWM(owm_api_key)
    mgr = owm.weather_manager()
    observation = mgr.weather_at_place(location)
    location_temperature = observation.weather.temperature(unit)['temp']
    weathernow = observation.weather.detailed_status 
    weather_info = {
        "location": location,
        "temperature": location_temperature,
        "unit": unit,
        "currentweather": weathernow,
        "forecast": ["sunny", "windy"], #fake, hard-coded values to show that we can do the obvious. :)
    }
    return json.dumps(weather_info)

Now, let's get a GPT completion with the function call. 

In [6]:
# courtesy OpenAI -- minor changes to code, the model, and required parameters 

def run_conversation():
    # Step 1: send the conversation and available functions to GPT
    messages = [{"role": "user", "content": userInput }]
    functions = [
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location", "unit"],
            },
        }
    ]
    response = openai.ChatCompletion.create(
        model="gpt-4-0613",  
        messages=messages,
        functions=functions,
        function_call="auto",  # auto is default, but we'll be explicit
    )
    response_message = response["choices"][0]["message"]

    # Step 2: check if GPT wanted to call a function
    if response_message.get("function_call"):
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_current_weather": get_current_weather,
        }  # only one function in this example, but you can have multiple
        function_name = response_message["function_call"]["name"]
        function_to_call = available_functions[function_name]
        function_args = json.loads(response_message["function_call"]["arguments"])
        print('Based on the location information GPT decided the temperature units should be ', function_args.get("unit"))
        print ('****************************************')
        function_response = function_to_call(
            location=function_args.get("location"),
            unit=function_args.get("unit"),
        )


        # Step 4: send the info on the function call and function response to GPT
        messages.append(response_message)  # extend conversation with assistant's reply
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response
        second_response = openai.ChatCompletion.create(
            model="gpt-4-0613",
            messages=messages,
        )  # get a new response from GPT where it can see the function response
        print('Response from the model:' , second_response.choices[0].message.content)
        print ('****************************************')
        return second_response
print ('****************************************')    
print('Completion Object:', run_conversation())
print ('****************************************')

****************************************
Based on the location information GPT decided the temperature units should be  celsius
****************************************
Response from the model: The current temperature in Jamaica is 24.61 degrees Celsius with moderate rain. The forecast predicts sunny and windy weather.
****************************************
Completion Object: {
  "id": "chatcmpl-86VByKSTzygDWOcMgMW4EofUVqSeI",
  "object": "chat.completion",
  "created": 1696560562,
  "model": "gpt-4-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The current temperature in Jamaica is 24.61 degrees Celsius with moderate rain. The forecast predicts sunny and windy weather."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 93,
    "completion_tokens": 24,
    "total_tokens": 117
  }
}
****************************************
